In [ ]:
import sys
import argparse
import os

sys.argv = ["view", "--config", "../../config/multi_task_cross_stitch.yaml"]

parser = argparse.ArgumentParser()
parser.add_argument("--config", type=str, required=True, help="Path to the config file")
args = parser.parse_args()

print(args.config)

In [ ]:
from dataloader import VOC08Attr
from torchvision.transforms import transforms
from config_experiments import config
from model import ObjectDetectionModel, AttributePredictionModel, CrossStitchNet
import torch
import numpy as np
import torch.onnx
import netron
from utils import set_device

In [ ]:
os.getcwd()
os.chdir("../../")
os.getcwd()

In [ ]:
transform_train = transforms.Compose(
    [
        transforms.Resize(size=config["transform"]["resize_values"]),
        transforms.ToTensor(),
        transforms.Normalize(
            mean=config["transform"]["mean"], std=config["transform"]["std"]
        ),
    ]
)
device = set_device(config["global"]["gpu_id"])

In [ ]:
model = CrossStitchNet(
    ObjectDetectionModel().alex,
    AttributePredictionModel().alex,
)

In [ ]:
model.eval()

data_train = VOC08Attr(transform=transform_train, train=True)

image, _, gt_class, gt_bbox, gt_attributes, ss_rois = data_train[0]

num_roi = ss_rois.shape[0]
ridx = np.zeros(num_roi).astype(int)
ridx = torch.from_numpy(ridx).unsqueeze(-1)

name_file = "src/multi_task_cross_stitch/model_cross.onnx"

torch.onnx.export(model, (image.unsqueeze(0), ss_rois, ridx), name_file)
netron.start(name_file)